In [1]:
__author__ = 'dmitry_sh'
from sklearn.externals import joblib


class SentimentClassifier(object):
    def __init__(self):
        self.model = joblib.load("../data/model.jbl")
        self.vectorizer = joblib.load("../data/vectorizer.jbl")
        self.classes_dict = {0: "negative", 1: "positive", -1: "prediction error"}

    @staticmethod
    def get_probability_words(probability):
        if probability < 0.55:
            return "neutral or uncertain"
        if probability < 0.7:
            return "probably"
        if probability > 0.95:
            return "certain"
        else:
            return ""

    def predict_text(self, text):
        try:
            vectorized = self.vectorizer.transform([text])
            return self.model.predict(vectorized)[0],\
                   self.model.predict_proba(vectorized)[0].max()
        except:
            print ("prediction error")
            return -1, 0.8

    def predict_list(self, list_of_texts):
        try:
            vectorized = self.vectorizer.transform(list_of_texts)
            return self.model.predict(vectorized),\
                   self.model.predict_proba(vectorized)
        except:
            print ('prediction error')
            return None

    def get_prediction_message(self, text):
        prediction = self.predict_text(text)
        class_prediction = prediction[0]
        prediction_probability = prediction[1]
        return self.get_probability_words(prediction_probability) + " " + self.classes_dict[class_prediction]

In [50]:
__author__ = 'dmitry_sh'
#from sentiment_classifier import SentimentClassifier
from codecs import open
import time
from flask import Flask, render_template, request
app = Flask(__name__)

print("Preparing classifier")
start_time = time.time()
classifier = SentimentClassifier()
print ("Classifier is ready")
print (time.time() - start_time, "seconds")

@app.route("/sentiment-demo", methods=["POST", "GET"])
def index_page(text="", prediction_message=""):
    if request.method == "POST":
        text = request.form["text"]
        logfile = open("ydf_demo_logs.txt", "a", "utf-8")
        print(text)
        print("<response>", file=logfile)
        print("<date_time>" + time.strftime('[%d/%m/%Y %H:%M:%S]') + "</date_time>", file=logfile)
        print("<text>", file=logfile)
        print(text, file=logfile)
        print("</text>", file=logfile)
        prediction_message = classifier.get_prediction_message(text)
        print('ОЦЕНКА:', prediction_message, '\n------------------')
        print("<prediction_message>" + prediction_message, "</prediction_message>" + file=logfile)
        print("</response>", file=logfile)
        logfile.close()
        
    return render_template('hello.html', text=text, prediction_message=prediction_message)


if __name__ == "__main__":
    app.run(host='0.0.0.0', port=80, debug=False)
    #app.run()

Preparing classifier


 * Running on http://0.0.0.0:80/ (Press CTRL+C to quit)


Classifier is ready
11.583807229995728 seconds


127.0.0.1 - - [09/Mar/2018 13:31:16] "POST /sentiment-demo HTTP/1.1" 200 -


хороший фильм
probably negative


...

127.0.0.1 - - [09/Mar/2018 13:31:49] "POST /sentiment-demo HTTP/1.1" 200 -


очень хороший фильм
probably negative


127.0.0.1 - - [09/Mar/2018 13:31:52] "POST /sentiment-demo HTTP/1.1" 200 -


очень хороший фильм
probably negative


127.0.0.1 - - [09/Mar/2018 13:40:08] "GET /mobilereview-demo HTTP/1.1" 404 -
127.0.0.1 - - [09/Mar/2018 13:40:19] "GET /mobilereview-demo HTTP/1.1" 404 -


In [80]:
time.strftime('[%d/%b/%Y %H:%M:%S]')

'[11/Mar/2018 11:29:37]'

In [79]:
time.strftime('%c')

'Sun Mar 11 11:18:16 2018'

In [6]:
__name__

'__main__'